In [1]:
import os
import analysis
import hrf_tools
import numpy as np
import nibabel as nb
from matplotlib import pyplot as plt  # Matlab-ish plotting commands

#import matplotlib
#matplotlib.rcParams['figure.figsize'] = (6, 4)
# this is run by run_datalad_run_encoding_model.sh
# "python ./run_encoding_model.py $sub $2"

/opt/miniconda-latest/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
parcels='../sourcedata/data/parcellations/combined_Q1-Q6_RelatedParcellation210.CorticalAreas_dil_Final_Final_Areas_Group_Colors.59k_fs_LR.dlabel.nii'
p_59k=nb.load(parcels)
p_59k_ind=np.array(p_59k.get_fdata())

In [3]:
axes = [p_59k.header.get_axis(i) for i in range(p_59k.ndim)]
label_axis, brain_model_axis = axes
#label_axis[0] #list the regions

In [4]:
def surf_data_from_cifti(data, axis, surf_name):
    assert isinstance(axis, nb.cifti2.BrainModelAxis)
    for name, data_indices, model in axis.iter_structures():  # Iterates over volumetric and surface structures
        if name == surf_name:                                 # Just looking for a surface
            data = data.T[data_indices]                       # Assume brainmodels axis is last, move it to front
            vtx_indices = model.vertex                        # Generally 1-N, except medial wall vertices
            surf_data = np.zeros((vtx_indices.max() + 1,) + data.shape[1:], dtype=data.dtype)
            surf_data[vtx_indices] = data
            return surf_data
    raise ValueError(f"No structure named {surf_name}")

In [5]:
volume= (p_59k,brain_model_axis)

In [6]:
volume

(<nibabel.cifti2.cifti2.Cifti2Image at 0x2b37df4547f0>,
 <nibabel.cifti2.cifti2_axes.BrainModelAxis at 0x2b37b3f43460>)

In [7]:
#a1,lbelt,mbelt,pbelt,RI (early)
#a4,a5,STSdp,STSda,STSvp,STSva,STGa,TA2
auditory_regions=[24,204,174,354,173,353,124,304,104,284,175,355,125,305,129,309,128,308,130,310,176,356,123,303,107,287]
early_auditory_regions=[24,204,174,354,173,353,124,304,104,284]

In [8]:
auditory_mask=np.zeros(p_59k_ind.shape)
for i in auditory_regions:
    for ii in np.where(p_59k_ind==i)[1]:
        auditory_mask[0,ii]=1
        
early_auditory_mask=np.zeros(p_59k_ind.shape)
for i in early_auditory_regions:
    for ii in np.where(p_59k_ind==i)[1]:
        early_auditory_mask[0,ii]=1

In [10]:
np.save('../sourcedata/data/parcellations/mmp_59k.npy',p_59k_ind )
np.save('../sourcedata/data/parcellations/early_auditory_59k.npy',early_auditory_mask )
np.save('../sourcedata/data/parcellations/auditory_59k.npy',auditory_mask )